In [1]:
import pandas as pd
from Acesso import Login
from Query import Query
from glob import glob

s=Login()

sql=Query(s.usuario,s.senha,s.database,s.server)

In [2]:
arquivos=glob('*.xlsx')

df=pd.read_excel(arquivos[-1])

In [3]:
df

,SKU,Produto,Qtde Caixa,B,C,D,E,F,FUNC,B2C,...,DISTRIBUIDORES,Dif.1,TABELA FUNCIONÁRIO,Dif.2,TABELA CONSUMIDOR FINAL,Dif.3,Desc C,Desc D,Desc E,Desc F
0,1074,"IQF MORANGO SANTA CATARINA(1,020 KG)",15,20.58,18.03,17.75,17.23,16.85,12.78,25.01,...,11.63,-6.12,8.41,-4.37,10.92,-14.09,0.1239,0.1375,0.1628,0.1812
1,1078,POLPINHA SUCO DE TANGERINA (100 GR),6,24.43,22.92,21.59,20.69,20.41,16.37,29.22,...,21.58,-0.01,16.36,-0.01,29.20,-0.02,0.0618,0.1163,0.1531,0.1646
2,9403,CONG. BATATA MCCAIN BINT 9MM (8X2KG),8,24.22,23.21,22.89,22.58,22.28,18.36,28.32,...,20.29,-2.60,16.27,-2.09,25.10,-3.22,0.0417,0.0549,0.0677,0.0801


In [9]:
#00000029,00000045,00000004,00000050

#alterar desconto

codigos=df['SKU'].unique().tolist()

conecta=sql.ConexaoSQL()

for c in codigos:
    
    try:
    
        desc=df.loc[df['SKU']==c,'Desc F'].max()
        
        querys=f"""

        UPDATE preco
        SET desc_prom={float(desc)},
        desc_max_prd={float(desc)}
        WHERE cd_prod={c} AND LTRIM(RTRIM(cd_tabela))='00000029'

        """

        sql.Salvar(querys,conecta)
        
        pass
    
    except Exception as erro:
        
        print(erro)
        
        pass
    
    #break
    
    pass


In [4]:
#alteração de preço massivo
tabelas_dict={'00000029':'B','00000045':'D','00000004':'FUNC','00000050':'B2C'}

codigos=df['SKU'].unique().tolist()

conecta=sql.ConexaoSQL()

for cod,tab in tabelas_dict.items():
    
    for c in codigos:
        
        preco=df.loc[df['SKU']==c,tab].max()
        
        #Unid. VDA
        unid=df.loc[df['SKU']==c,'Unid. VDA'].max()
        
        #caixa Qtde Caixa
        caixa=df.loc[df['SKU']==c,'Qtde Caixa'].max()
        
        preco=preco if unid!='CX' else round(preco/caixa,2)

        querys=f"""

        UPDATE preco
        SET vl_preco={float(preco)}
        WHERE cd_prod={c} AND LTRIM(RTRIM(cd_tabela))='{cod}'

        """

        sql.Salvar(querys,conecta)

        #break

        pass
    
    print(f'Tabela {tab} alterada com sucesso!')
    
    pass


Tabela B alterada com sucesso!
Tabela D alterada com sucesso!
Tabela FUNC alterada com sucesso!
Tabela B2C alterada com sucesso!
